
Facial Recognition using PyTorch and OpenCV

https://ritik12.medium.com/facial-recognition-using-pytorch-and-opencv-467c4e41d1f


Machine Learning - Face Recognition CNN Pytorch.ipynb
https://github.com/rubencg195/Pytorch-Tutorials/blob/master/Machine%20Learning%20-%20Face%20Recognition%20CNN%20Pytorch.ipynb



Face Recognition Using Pytorch
https://github.com/timesler/facenet-pytorch

Face Landmarks Detection With PyTorch

https://towardsdatascience.com/face-landmarks-detection-with-pytorch-4b4852f5e9c4



다중입력 deep neural network
https://rosenfelder.ai/multi-input-neural-network-pytorch/



Understanding dimensions in PyTorch
https://towardsdatascience.com/understanding-dimensions-in-pytorch-6edf9972d3be

In [1]:
from torch.optim import lr_scheduler
from torch.nn.init import *
from torchvision import transforms, utils, datasets, models
from pathlib import Path
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import pandas as pd
import numpy as np
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import datasets
from torchvision.transforms import ToTensor

from FaceFeatureDataset import FaceFeatureDataset

# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


c:\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using cuda device


In [2]:
batch_size = 10
epochs = 10

In [3]:
linear_layer_2d = nn.Linear(in_features=68, out_features=128)
flatten = nn.Flatten()
input_2d = torch.randn(2, 2, 68) # n, channel, features
#print(input_2d)
print(input_2d.shape)
output_2d = linear_layer_2d(input_2d)
print('output_2d' , output_2d)
#print(output_2d.size())

#output_1d = flatten(input_2d)
#print(output_1d.size())
#linear_layer_2d(output_1d)
#print("1D ",  output_1d)

#test_sequence = nn.Sequential(nn.Linear(64, 32), nn.ReLU(), nn.Linear(32,10))
#out = test_sequence(input_2d)
#print(out)



torch.Size([2, 2, 68])
output_2d tensor([[[-6.1806e-01, -7.7384e-02,  1.9195e-01, -1.0017e+00,  1.2797e-01,
          -7.1703e-02,  1.4624e-01, -7.8517e-01, -5.6697e-01, -1.0013e-01,
           4.2946e-01, -2.2345e-01,  1.2505e-01, -4.7891e-01,  6.8265e-02,
          -8.6848e-02,  2.4225e-01,  1.6318e-01,  1.7623e-01,  5.0248e-02,
          -5.3683e-02,  4.1974e-02, -1.2864e-01, -5.5282e-01, -4.7931e-01,
           2.7521e-01, -2.0208e-01,  6.2883e-01,  9.1397e-02, -1.9679e-01,
          -2.5712e-01,  4.7575e-01,  1.7274e-02,  5.8329e-01,  7.9755e-01,
          -1.6319e-02,  4.3476e-01, -5.5405e-01, -2.6931e-01, -5.3151e-01,
          -3.2860e-01,  1.3785e-01, -5.0351e-02,  1.0057e+00,  1.7901e-01,
          -7.3359e-01,  8.3160e-02,  3.9779e-01,  5.1801e-01,  6.9527e-02,
          -3.0596e-01,  6.8492e-01, -6.4788e-04,  6.8850e-01, -3.3462e-02,
           5.4772e-01, -1.8067e-01, -5.5819e-03,  2.5021e-01, -4.6806e-01,
           5.4527e-01,  8.3101e-02, -2.0863e-01,  3.0555e-01,  7.79

In [4]:
'''
testload = pd.read_csv("./outimg/Train/facefeature.csv")
landmarks = np.array(testload.iloc[0, 1:])
landmarks = landmarks.astype('float').reshape(-1, 2)
print(landmarks)

dataiter = iter(testload)
landmark = next(dataiter)
print(landmark)
# print(landmarks.shape)
'''

'\ntestload = pd.read_csv("./outimg/Train/facefeature.csv")\nlandmarks = np.array(testload.iloc[0, 1:])\nlandmarks = landmarks.astype(\'float\').reshape(-1, 2)\nprint(landmarks)\n\ndataiter = iter(testload)\nlandmark = next(dataiter)\nprint(landmark)\n# print(landmarks.shape)\n'

In [5]:
# Create DataLoader 

training_data = FaceFeatureDataset(feature_file="./outimg/Train/facefeature.csv", label_file="./Dataset/Train/csv/train.csv")
test_data = FaceFeatureDataset(feature_file="./outimg/Test/facefeature.csv", label_file="./Dataset/Test/csv/test.csv")

train_loader = DataLoader(training_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# 데이터 로드 확인 
for X, y in train_loader:
    print(f"Shape of X [N, F, C]: {X.shape}") # N , Channel, H= width W = height
    print(f"Shape of Tensor y: {y.shape} {y.dtype}")       
    break

n_total_steps = len(train_loader)
# print(f'Traing dat length {n_total_steps}')
# print(y)
print('X type', X.dtype)
print('y type', y.dtype)


Shape of X [N, F, C]: torch.Size([10, 136])
Shape of Tensor y: torch.Size([10, 33]) torch.float32
X type torch.float32
y type torch.float32


In [6]:
#
#dataiter = iter(train_loader)
#landmark, labels = next(dataiter)
#print("landmark Shape ", landmark.shape)
#print(landmark)
#flatten = nn.Flatten()
#linear1 = nn.Linear(68, 32)
#x = flatten(landmark)
#print(x)
#print(x.size())
# print(landmark.shape)
# print('linear1', x.shape)


In [7]:

num_classes = 33

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        #self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(68 * 2, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
            #nn.ReLU(),
        )
    

    def forward(self, x):
        # x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)


NeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=136, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=33, bias=True)
  )
)


In [8]:
#criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)


In [9]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        pred = model(X)
        # print(pred)
        loss = loss_fn(pred, y)
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # print('batch',  batch)
        if batch % 10 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [10]:
# Training
epochs = 1000

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, criterion, optimizer)    
print("Done!")

Epoch 1
-------------------------------
loss: 0.280875  [    0/  100]
Epoch 2
-------------------------------
loss: 0.265239  [    0/  100]
Epoch 3
-------------------------------
loss: 0.217329  [    0/  100]
Epoch 4
-------------------------------
loss: 0.198985  [    0/  100]
Epoch 5
-------------------------------
loss: 0.165768  [    0/  100]
Epoch 6
-------------------------------
loss: 0.122368  [    0/  100]
Epoch 7
-------------------------------
loss: 0.086326  [    0/  100]
Epoch 8
-------------------------------
loss: 0.062391  [    0/  100]
Epoch 9
-------------------------------
loss: 0.040026  [    0/  100]
Epoch 10
-------------------------------
loss: 0.032136  [    0/  100]
Epoch 11
-------------------------------
loss: 0.032590  [    0/  100]
Epoch 12
-------------------------------
loss: 0.032267  [    0/  100]
Epoch 13
-------------------------------
loss: 0.032503  [    0/  100]
Epoch 14
-------------------------------
loss: 0.033684  [    0/  100]
Epoch 15
------

In [11]:
PATH = './cifar_net.pth'
torch.save(model.state_dict(), PATH)

In [12]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    print('test size', size )
    # num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0,0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            loss = loss_fn(pred, y)            
            print('pred =', pred)
            #print('loss', loss)
            #print('real', y)



In [13]:

test(test_loader, model, criterion)

print("Done !")

test size 100
pred = tensor([[0.4884, 0.4862, 0.4923, 0.5085, 0.5192, 0.5185, 0.4763, 0.5034, 0.5230,
         0.5138, 0.5209, 0.5012, 0.4917, 0.5166, 0.4849, 0.4951, 0.5101, 0.5320,
         0.4967, 0.5214, 0.4975, 0.5065, 0.4967, 0.5018, 0.5170, 0.4897, 0.5260,
         0.4863, 0.4821, 0.5072, 0.5250, 0.5167, 0.4802],
        [0.4920, 0.4882, 0.4950, 0.5091, 0.5206, 0.5220, 0.4753, 0.5012, 0.5247,
         0.5127, 0.5220, 0.5015, 0.4946, 0.5160, 0.4855, 0.4925, 0.5120, 0.5376,
         0.4959, 0.5207, 0.4994, 0.5097, 0.4957, 0.5017, 0.5200, 0.4923, 0.5276,
         0.4896, 0.4837, 0.5108, 0.5245, 0.5171, 0.4827],
        [0.4885, 0.4850, 0.4926, 0.5069, 0.5180, 0.5209, 0.4739, 0.5000, 0.5215,
         0.5102, 0.5197, 0.5001, 0.4921, 0.5144, 0.4830, 0.4909, 0.5100, 0.5343,
         0.4944, 0.5195, 0.4983, 0.5068, 0.4944, 0.4994, 0.5174, 0.4898, 0.5252,
         0.4863, 0.4819, 0.5068, 0.5226, 0.5148, 0.4809],
        [0.4870, 0.4815, 0.4897, 0.5044, 0.5160, 0.5181, 0.4707, 0.4955, 0.5